In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt


In [ ]:

# Load data from Excel sheet
def load_data_from_excel(file_path):
    df = pd.read_excel(file_path)
    return df

# Sample data preparation
data_path = '/content/consolidated data.xlsx'
df = load_data_from_excel(data_path)


In [ ]:
# Splitting data into features (X) and target variables (y)
X = df[['speed', 'distance', 'remainingrange', 'batteryvoltage', 'batterycurrent', 'cellmaxvoltage', 'cellminvoltage', 'mcu_dcvoltage', 'mcu_dccurrent', 'mcu_acrmscurrent', 'mcu_speed', 'mcu_temperature']].values
y_soh = df['batterysoh'].values.reshape(-1, 1)
y_soc = df['batterysoc'].values.reshape(-1, 1)

# Scaling features and target variables
scaler_X = MinMaxScaler()
scaler_y_soh = MinMaxScaler()
scaler_y_soc = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled_soh = scaler_y_soh.fit_transform(y_soh)
y_scaled_soc = scaler_y_soc.fit_transform(y_soc)

# Convert data to PyTorch tensors
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_soh_tensor = torch.tensor(y_scaled_soh, dtype=torch.float32)
y_soc_tensor = torch.tensor(y_scaled_soc, dtype=torch.float32)


In [ ]:
# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.fc_soh = nn.Linear(hidden_size, output_size)
        self.fc_soc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x.view(len(x), 1, -1))
        lstm_out = lstm_out.view(-1, self.hidden_size)  # Flatten the LSTM output
        output_soh = self.fc_soh(lstm_out)
        output_soc = self.fc_soc(lstm_out)
        return output_soh, output_soc



In [ ]:
# Initialize the model
input_size = X_tensor.shape[1]
hidden_size = 50
output_size = 1  # Output size for SoH and SoC
model = LSTMModel(input_size, hidden_size, output_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Training the model
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    output_soh, output_soc = model(X_tensor)
    loss_soh = criterion(output_soh, y_soh_tensor)
    loss_soc = criterion(output_soc, y_soc_tensor)
    loss = loss_soh + loss_soc
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], SoH Loss: {loss_soh.item():.4f}, SoC Loss: {loss_soc.item():.4f}')


Epoch [10/100], SoH Loss: 0.0712, SoC Loss: 0.2871
Epoch [20/100], SoH Loss: 0.0061, SoC Loss: 0.0511
Epoch [30/100], SoH Loss: 0.0076, SoC Loss: 0.0538
Epoch [40/100], SoH Loss: 0.0068, SoC Loss: 0.0555
Epoch [50/100], SoH Loss: 0.0056, SoC Loss: 0.0498
Epoch [60/100], SoH Loss: 0.0057, SoC Loss: 0.0500
Epoch [70/100], SoH Loss: 0.0055, SoC Loss: 0.0489
Epoch [80/100], SoH Loss: 0.0054, SoC Loss: 0.0483
Epoch [90/100], SoH Loss: 0.0054, SoC Loss: 0.0478
Epoch [100/100], SoH Loss: 0.0053, SoC Loss: 0.0471


In [ ]:
import torch

# Load model architecture information
model_info = torch.load("lstm_model_info.pth")

# Reinitialize the model with the same architecture
model = LSTMModel(model_info['input_size'], model_info['hidden_size'], model_info['output_size'])

# Load model state dictionary
model.load_state_dict(torch.load("lstm_model.pth"))
model.eval()  # Set the model to evaluation mode


LSTMModel(
  (lstm): LSTM(12, 50)
  (fc_soh): Linear(in_features=50, out_features=1, bias=True)
  (fc_soc): Linear(in_features=50, out_features=1, bias=True)
)

In [ ]:
model = torch.load('LSTM_MODEL.pth')

In [ ]:

# Manually input new data
new_data = np.array([[
    float(input("Enter speed: ")),
    float(input("Enter distance: ")),
    float(input("Enter remaining range: ")),
    float(input("Enter battery voltage: ")),
    float(input("Enter battery current: ")),
    float(input("Enter cell max voltage: ")),
    float(input("Enter cell min voltage: ")),
    float(input("Enter MCU DC voltage: ")),
    float(input("Enter MCU DC current: ")),
    float(input("Enter MCU AC rms current: ")),
    float(input("Enter MCU speed: ")),
    float(input("Enter MCU temperature: "))
]])

# Scale the input data
new_data_scaled = scaler_X.transform(new_data)
new_data_tensor = torch.tensor(new_data_scaled, dtype=torch.float32)

# Predicting SoH and SoC for new data
with torch.no_grad():
    predicted_soh, predicted_soc = model(new_data_tensor)
    predicted_soh_unscaled = scaler_y_soh.inverse_transform(predicted_soh.reshape(-1, 1))
    predicted_soc_unscaled = scaler_y_soc.inverse_transform(predicted_soc.reshape(-1, 1))



Enter speed: 1
Enter distance: 2
Enter remaining range: 12
Enter battery voltage: 4
Enter battery current: 5
Enter cell max voltage: 2
Enter cell min voltage: 3
Enter MCU DC voltage: 4
Enter MCU DC current: 4
Enter MCU AC rms current: 21
Enter MCU speed: 32
Enter MCU temperature: 23


In [ ]:
# Print the predicted SOC and SOH
print(f'Predicted SOH: {predicted_soh_unscaled.item()}')
print(f'Predicted SOC: {predicted_soc_unscaled.item()}')

# Calculate RMSE for SOH
rmse_soh = torch.sqrt(criterion(predicted_soh, y_soh_tensor)).item()
print(f'RMSE for SOH: {rmse_soh:.4f}')

# Calculate RMSE for SOC
rmse_soc = torch.sqrt(criterion(predicted_soc, y_soc_tensor)).item()
print(f'RMSE for SOC: {rmse_soc:.4f}')

Predicted SOH: 10.133331030607224
Predicted SOC: 6.049779057502747
RMSE for SOH: 0.3294
RMSE for SOC: 0.6254


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1241, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
